# Supplemental analysis - including all regions of interest in activity flow estimate

In [1]:
import numpy as np
import pandas as pd
import functions as mf # my functions
from scipy import stats
import pingouin as pg
#from statsmodels.stats.multitest import multipletests

## critical variables
# parcellation
PARC = 'cabn'

# subjects
subj_df = pd.read_csv('subject_list.txt', sep='\t', index_col = 0, header = 0)

# out directories
results_dir = '/projects/f_mc1689_1/ClinicalActFlow/data/results/N=93/'
figure_dir = '/projects/f_mc1689_1/ClinicalActFlow/docs/figures/N=93/'

# which connectivity type to use
fc_task = 'multitask-no-scap'
fc_method = 'pc_multregconn_100'

# task to analyze
task = 'scap'

# groups to analyze
groups = ['CTRL','SCHZ']

/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/projects/f_mc1689_1/AnalysisTools/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/home/ljh118/.local/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.2, the latest is 0.3.8.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


# Prep data

In [2]:
#load activations
activity,activity_all = mf.load_activity(subj_df,PARC=PARC,TASKS=[task])
network_order,_,network_def,networks = mf.get_network_info(PARC)
n_roi = activity['scap']['CTRL'].shape[0]

# load fc data
fc,fc_all = mf.load_fc(subj_df,TASKS=[fc_task],PARC=PARC,fc_method=fc_method)
network_order,network_cols,network_def,networks = mf.get_network_info(PARC,subcortical_split=True)

Task| multitask-no-scap Data loaded: 100.0 %


# Run actflow without holding out any regions

In [3]:
# generate activity flow predictions seperately in each group
task = 'scap'
af_file = 'actflow-' + fc_task + '-' + fc_method + '.pickle'

predicted_activity = {}
predicted_activity[task] = {}
actFlowWeights = {}
actFlowWeights[task] = {}

for group in ['CTRL','SCHZ']:
    print('Running act flow in',group)
    fc_data = fc[fc_task][group].copy()

    actPredVector = np.zeros((np.shape(activity[task][group])))
    n_nodes =  np.shape(actPredVector)[0]
    n_conditions = np.shape(actPredVector)[1]
    n_subs = np.shape(actPredVector)[2]
    act_weights_mat = np.zeros((n_nodes,n_nodes,n_conditions,n_subs))

    for condition in range(n_conditions):
        act_data = activity[task][group][:,condition,:].copy()

        for subj in range(np.shape(fc_data)[2]):
            actPredVector[:,condition,subj],act_weights_mat[:,:,condition,subj] = mf.actflowcalc_hold_out_roi(act_data[:,subj],fc_data[:,:,subj])

    predicted_activity[task][group] = actPredVector
    actFlowWeights[task][group] = act_weights_mat

print('Actflow finished')

Running act flow in CTRL
Running act flow in SCHZ
Actflow finished


## Perform stats as usual

In [5]:
from scipy.stats import ttest_ind
# do stats on the activity flow predictions
r = {}
MAE = {}
MAPE = {}
Rsqr = {}
for group in ['CTRL','SCHZ']:
    # do the same contrast
    real = np.mean(activity['scap'][group][:,6:12,:],axis=1) - np.mean(activity['scap'][group][:,0:6,:],axis=1)
    pred = np.mean(predicted_activity['scap'][group][:,6:12,:],axis=1) - np.mean(predicted_activity['scap'][group][:,0:6,:],axis=1)
    
    # do actflow statistics
    r[group] = []
    MAE[group] = []
    MAPE[group] = []
    Rsqr[group] = []
    r[group],rs,MAE[group],MAPE[group],Rsqr[group] = mf.actflow_tests(real,pred,normalise=False)
    
# compare the groups
print('between groups: r t-test')
print('\t',ttest_ind(r['CTRL'],r['SCHZ'],equal_var=False))
print('between groups: MAE t-test')
print('\t',ttest_ind(MAE['CTRL'],MAE['SCHZ'],equal_var=False))
print('between groups: Rsqr t-test')
print('\t',ttest_ind(Rsqr['CTRL'],Rsqr['SCHZ'],equal_var=False))

Mean r across subjs: 0.632 |1samp t: 57.37 p: 0.0
Mean MAE  across subjs: 0.619
Mean MAPE  across subjs: 357.226
Mean R^2  across subjs: 0.395 |1samp t: 26.05 p: 0.0
Mean r across subjs: 0.6 |1samp t: 31.41 p: 0.0
Mean MAE  across subjs: 0.606
Mean MAPE  across subjs: 420.539
Mean R^2  across subjs: 0.352 |1samp t: 13.6 p: 0.0
between groups: r t-test
	 Ttest_indResult(statistic=1.476369251021821, pvalue=0.14510086571181408)
between groups: MAE t-test
	 Ttest_indResult(statistic=0.5857193234237867, pvalue=0.5606582862776508)
between groups: Rsqr t-test
	 Ttest_indResult(statistic=1.4548232159257706, pvalue=0.1508833688116412)


# target region results

In [8]:
from statsmodels.stats.multitest import multipletests

#defined in actual dataset
roi_list = [56, 181, 284, 346] 

In [9]:
# between groups t-test, FDR corrected - replication of the original analysis
x = np.mean(predicted_activity[task]['CTRL'][:,6:12,:],axis=1) - np.mean(predicted_activity[task]['CTRL'][:,0:6,:],axis=1)
y = np.mean(predicted_activity[task]['SCHZ'][:,6:12,:],axis=1) - np.mean(predicted_activity[task]['SCHZ'][:,0:6,:],axis=1)

p = []
for roi in roi_list:
    res = pg.ttest(x[roi,:],y[roi,:])
    display(res.round(3))
    p.append(res['p-val'].values[0])

h,padj,_,_ = multipletests(np.array(p),method='bonferroni')
print('Bonferonni adjusted p-vals:',np.round(padj,3))
print('')

# test accuracy within each roi
r,MAE,MAPE,Rsqr = mf.roi_level_accuracy(activity,predicted_activity,roi_list)
print('Correlation within each roi=',r)
print('MAE within each roi=',MAE)
print('MAPE within each roi=',MAPE)
print('Rsqr within each roi=',Rsqr)

df_roi_metrics = pd.DataFrame()
df_roi_metrics['actflow-full'] = r
df_roi_metrics['metric'] = 'r'

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-3.084,95.2,two-sided,0.003,"[-0.56, -0.12]",0.509,13.276,0.73


,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-1.679,72.49,two-sided,0.097,"[-0.41, 0.03]",0.31,0.725,0.348


,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-3.456,82.51,two-sided,0.001,"[-0.67, -0.18]",0.604,37.425,0.863


,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-3.504,88.47,two-sided,0.001,"[-0.56, -0.15]",0.596,43.113,0.854


Bonferonni adjusted p-vals: [0.011 0.39  0.003 0.003]

Correlation within each roi= [0.78322169 0.77330275 0.81378934 0.78510602]
MAE within each roi= [1.02287739 1.22178369 1.02205485 0.98208673]
MAPE within each roi= [ 144.27975595  391.05931329 1204.0853627   306.65412202]
Rsqr within each roi= [0.60979724 0.55404085 0.65859907 0.60052566]
